In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from ascalar import Scalar

NameError: name 'Scalar' is not defined

In [3]:
# check value
a = Scalar(5)

# check repr
print(a)

Scalar(value=5)


In [11]:
# check operations
b = Scalar(3)

a_plus_b = a + b
assert a_plus_b == Scalar(8)

a_minus_b = a - b
assert a_minus_b == Scalar(2)

a_times_b = a * b
assert a_times_b == Scalar(15)

a_div_b = a / b
print(a_div_b)
print(5/3)
assert a_div_b == Scalar(5/3)



NameError: name 'math' is not defined